# OpenStreetMap Project #

Include:
- Snippets of code
- Problematic tags
- Visualizations if appropriate

## Section 1: Problems encountered in the map

### 1a: Audit of 'tag' elements

To develop a rough sense of the data contained in the 'tag' elements of nodes, ways, and relations in the OSM file, a script was run to compile and count each unique 'key' in the tag elements. In total, 446 unique keys were present. After performing a cursory scan of the keys by eye, at least three large clusters became evident: (i) tags with 'tiger' data, (ii) tags with 'gnis' data, and (iii) tags labeled with 'fixme'. From the lessons, the possibility of keys with problematic characters was also evident. Consequently, the tag elements were broken down into the following five categories and counted:

problem: 1  
fixme: 36  
gnis: 1873  
tiger: 61901  
other: 90291  







## Section 2: Overview of the data

## Section 3: Other ideas about the dataset

Notes:
 - Aggregate keys once for efficiency (done)

In [34]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re

filename = 'Rochester.osm'

def is_node(element):
    return element.tag == 'node'

def is_way(element):
    return element.tag == 'way'

def is_relation(element):
    return element.tag == 'relation'

def is_tag(element):
    return element.tag == 'tag'

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)
        
def iter_elements(filename, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = ET.iterparse(filename, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [35]:
def aggregate_tag_keys(filename):
    keys = defaultdict(int)
    for element in iter_elements(filename):
        for subelement in element:
            if (subelement.tag == 'tag') and ('k' in subelement.attrib):
                keys[subelement.get('k')] += 1
    return keys

def find_problem_tags(filename):
    problem_keys = defaultdict(int)
    problem_chars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
    keys = aggregate_tag_keys(filename)
    for key in keys:
        if problem_chars.search(key):
            problem_keys[key] += 1
    return problem_keys
    
def categorize_tags(filename):
    problem_chars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
    key_categories = {'fixme':0, 'tiger':0, 'gnis':0, 'problem':0, 'other':0}
    keys = aggregate_tag_keys(filename)
    for key in keys:
        if problem_chars.search(key):
            key_categories['problem'] += keys[key]
        elif ('FIXME' in key) or ('fixme' in key):
            key_categories['fixme'] += keys[key]
        elif 'tiger' in key:
            key_categories['tiger'] += keys[key]
        elif 'gnis' in key:
            key_categories['gnis'] += keys[key]
        else:
            key_categories['other'] += keys[key]
    return key_categories

print_sorted_dict(categorize_tags(filename))

fixme: 36
gnis: 1873
other: 90291
problem: 1
tiger: 61901


In [36]:
def aggregate_addr_tags(filename):
    addr_keys = defaultdict(int)
    keys = aggregate_tag_keys(filename)
    for key in keys:
        if 'addr' in key:
            addr_keys[key] += keys[key]
    return addr_keys

addr_keys = aggregate_addr_tags(filename)
print_sorted_dict(addr_keys)

addr:city: 2727
addr:city_1: 2
addr:country: 1992
addr:floor: 2
addr:floot: 1
addr:housename: 31
addr:housenumber: 3144
addr:housenumber_1: 33
addr:housenumber_2: 4
addr:housenumber_3: 4
addr:housenumber_4: 4
addr:housenumber_5: 4
addr:place: 1
addr:postcode: 2799
addr:province: 1
addr:state: 2410
addr:street: 3249
addr:street_1: 3
addr:street_2: 1
addr:street_3: 1
addr:unit: 3
address: 14


In [11]:
def addr_key_w_num(filename):
    addr_w_num = defaultdict(int)
    has_num = re.compile(r'^(addr:)\w*[0-9]$')
    for _, element in ET.iterparse(filename, events=('start',)):
        if is_tag(element) and ('k' in element.attrib):
            key = element.get('k')
            if has_num.search(key):
                addr_w_num[key] += 1
    return addr_w_num
print_sorted_dict(addr_key_w_num(filename))

addr:city_1: 2
addr:housenumber_1: 33
addr:housenumber_2: 4
addr:housenumber_3: 4
addr:housenumber_4: 4
addr:housenumber_5: 4
addr:street_1: 3
addr:street_2: 1
addr:street_3: 1


In [10]:
def count_addr_keys(filename):
    addr_keys = defaultdict(int)
    has_addr = re.compile(r'^(addr:)')
    for _, element in ET.iterparse(filename):
        if is_tag(element) and ('k' in element.attrib):
            key = element.get('k')
            if has_addr.search(key):
                addr_keys[key] += 1
    return addr_keys
print_sorted_dict(count_addr_keys(filename))

def weird_one(filename):
    weird_key = "address"
    for _, element in ET.iterparse(filename):
        if is_tag(element) \
        and ('k' in element.attrib) \
        and (element.get('k') == weird_key):
            print element.get('v')                                                 
print '*****'
weird_one(filename)

addr:city: 2727
addr:city_1: 2
addr:country: 1992
addr:floor: 2
addr:floot: 1
addr:housename: 31
addr:housenumber: 3144
addr:housenumber_1: 33
addr:housenumber_2: 4
addr:housenumber_3: 4
addr:housenumber_4: 4
addr:housenumber_5: 4
addr:place: 1
addr:postcode: 2799
addr:province: 1
addr:state: 2410
addr:street: 3249
addr:street_1: 3
addr:street_2: 1
addr:street_3: 1
addr:unit: 3
*****
111 West Elm Street, East Rochester, NY 14445
971 South Avenue, Rochester, NY 14620
809 Monroe Avenue, Rochester, NY 14607
12 Bronson Avenue, Rochester, NY 14608
611 N. Winton Road, Rochester, NY 14609
956 Lyell Avenu, Rochester, NY 14606
939 Bay Street, Rochester, NY 14609
851 Joseph Avenue, Rochester, NY 14621
1111 Dewey Avenue, Rochester, NY 14613
2180 Ridge Road East, Rochester, NY 14622
45 Cooper Road, Rochester, NY 14617
2780 Dewey Avenue, Rochester, NY 14616
3615 Lake Avenue, Rochester, NY 14612
115 South Avenue, Rochester, NY 14604


In [9]:
def aggregate_street_abbrevs(filename):
    streets = defaultdict(int)
    street_tag = re.compile(r'^(addr:street)\w*')
    street_name = re.compile(r'\b\w+\b$')
    for _, element in ET.iterparse(filename):
        if is_tag(element) and ('k' in element.attrib):
            key = element.get('k')
            if street_tag.search(key):
                street = street_name.search(element.get('v'))
                if street:
                    streets[street.group()] += 1
                    if street.group() in ('N', 'S', 'E', 'W', 'PW'):
                        print element.get('v')
    return streets

print_sorted_dict(aggregate_street_abbrevs(filename))
            

Ridge Road W
Winton Road N
GBC PW
2: 1
92: 1
Apartment: 1
ave: 1
Ave: 7
Avenue: 531
Bl: 1
Blvd: 7
Boulevard: 12
Bridge: 6
Cir: 1
Circle: 138
Court: 60
Ct: 1
Dr: 9
Drive: 951
East: 16
Green: 5
Landing: 1
Lane: 428
line: 1
Manor: 9
Market: 1
N: 1
North: 3
NY: 1
Oaks: 1
Park: 4
Parkway: 41
Passage: 20
Pkwy: 1
Place: 12
PW: 1
Rd: 4
Reserve: 1
Road: 683
South: 25
Square: 1
St: 5
Stree: 1
Street: 111
Trail: 46
Villas: 10
W: 1
Way: 45
West: 41
Woods: 2


In [105]:
street = '10 Smith Dr'

def fix_street_abbrevs(street):
    mapping = {
        'ave': 'Avenue',
        'Ave': 'Avenue',
        'Avenu': 'Avenue',
        'Bl': 'Boulevard',
        'Blvd': 'Boulevard',
        'Cir': 'Circle',
        'Ct': 'Court',
        'Dr': 'Drive',
        'line': 'Line',
        'Pkwy': 'Parkway',
        'PW': 'Parkway',
        'Rd': 'Road',
        'St': 'Street',
        'Stree': 'Street',
        'N': 'North',
        'S': 'South',
        'E': 'East',
        'W': 'West'
    }
    
    elements = street.split()
    for i in range(len(elements)):
        if elements[i] in mapping:
            elements[i] = mapping[elements[i]]
    updated_street = ' '.join(elements)
    return updated_street

print fix_street_abbrevs(street)

10 Smith Drive


In [111]:
def aggregate_zips(filename):
    zips = defaultdict(int)
    for _, element in ET.iterparse(filename):
        if is_tag(element) \
        and ('k' in element.attrib) \
        and (element.get('k') == 'addr:postcode'):
            key = element.get('v')
            zips[key] += 1
    return zips

print_sorted_dict(aggregate_zips(filename))

1--: 1
14445: 17
14450: 1559
14526: 4
14534: 98
14580: 28
14604: 17
14605: 3
14606: 17
14607: 40
14608: 1
14609: 12
14610: 36
14611: 1
14612: 15
14613: 3
14614: 9
14615: 7
14616: 86
14617: 7
14617-1822: 1
14618: 371
14620: 22
14620-1327: 1
14621: 10
14622: 5
14623: 340
14624: 31
14624-4721: 1
14625: 7
14626: 41
14627: 3
14692: 1
14694: 3
West Main Street: 1


## Export to database ##

In [ ]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

OSM_PATH = "Rochester.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']




def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in node_attr_fields:
                node_attribs[attrib] = element.attrib[attrib]
        for subtag in element:
            temp_attrib = {}
            if subtag.tag == 'tag':
                #print subtag.attrib
                temp_attrib['id'] = node_attribs['id']
                temp_attrib['type'] = default_tag_type
                temp_attrib['value'] = subtag.get('v')
                key = subtag.get('k')
                if problem_chars.search(key):
                    continue
                elif LOWER_COLON.search(key):
                    key_segments = key.split(':')
                    temp_attrib['type'] = key_segments[0]
                    temp_attrib['key'] = ':'.join(key_segments[1:])
                else:
                    temp_attrib['key'] = key
                #print temp_attrib
                tags.append(temp_attrib)
    if element.tag == 'way':
        for attrib in element.attrib:
            if attrib in way_attr_fields:
                way_attribs[attrib] = element.attrib[attrib]
        print way_attribs
        i = 0
        for subtag in element:
            temp_attrib = {}
            if subtag.tag == 'tag':
                #print subtag.attrib
                temp_attrib['id'] = way_attribs['id']
                temp_attrib['type'] = default_tag_type
                temp_attrib['value'] = subtag.get('v')
                key = subtag.get('k')
                if problem_chars.search(key):
                    continue
                elif LOWER_COLON.search(key):
                    key_segments = key.split(':')
                    temp_attrib['type'] = key_segments[0]
                    temp_attrib['key'] = ':'.join(key_segments[1:])
                else:
                    temp_attrib['key'] = key
                print temp_attrib
                tags.append(temp_attrib)
            if subtag.tag == 'nd':
                temp_attrib['id'] = way_attribs['id']
                temp_attrib['node_id'] = subtag.get('ref')
                temp_attrib['position'] = i
                way_nodes.append(temp_attrib)
                i += 1
                print temp_attrib
            
    
    

    if element.tag == 'node':
        #print 'node:', node_attribs, 'node_tags:', tags
        #pprint.pprint({'node': node_attribs, 'node_tags': tags})
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        #pprint.pprint({'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags})
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

In [1]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

OSM_PATH = "Rochester.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']



def fix_street_abbrevs(street):
    '''Expand abbreviations in street names'''
    
    mapping = {
        'ave': 'Avenue',
        'Ave': 'Avenue',
        'Avenu': 'Avenue',
        'Bl': 'Boulevard',
        'Blvd': 'Boulevard',
        'Cir': 'Circle',
        'Ct': 'Court',
        'Dr': 'Drive',
        'line': 'Line',
        'Pkwy': 'Parkway',
        'PW': 'Parkway',
        'Rd': 'Road',
        'St': 'Street',
        'Stree': 'Street',
        'N': 'North',
        'S': 'South',
        'E': 'East',
        'W': 'West'
    }
    
    elements = street.split()
    for i in range(len(elements)):
        if elements[i] in mapping:
            elements[i] = mapping[elements[i]]
    updated_street = ' '.join(elements)
    return updated_street


def fix_zipcode(zipcode):
    '''Check the zipcode for the proper format'''
    
    zipformat = re.compile(r"(^[0-9]{5})(-[0-9]{4})?")
    if zipformat.match(zipcode):
        return zipcode
    else:
        return 'fixme'


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []

    # If the element is a node, extract the appropriate tags with valid keys
    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in node_attr_fields:
                node_attribs[attrib] = element.attrib[attrib]
        for child in element:
            temp_attrib = {}
            if child.tag == 'tag':
                key = child.get('k')
                if problem_chars.search(key):
                    continue
                elif LOWER_COLON.search(key):
                    key_segments = key.split(':')
                    temp_attrib['type'] = key_segments[0]
                    temp_attrib['key'] = ':'.join(key_segments[1:])
                else:
                    temp_attrib['key'] = key
                temp_attrib['id'] = node_attribs['id']
                temp_attrib['type'] = default_tag_type
                temp_attrib['value'] = child.get('v')
                tags.append(temp_attrib)
                
    # If the element is a way, extract the appropriate tags with valid keys            
    if element.tag == 'way':
        for attrib in element.attrib:
            if attrib in way_attr_fields:
                way_attribs[attrib] = element.attrib[attrib]
        i = 0
        for child in element:
            temp_attrib = {}
            if child.tag == 'tag':
                key = child.get('k')
                if problem_chars.search(key):
                    continue
                elif LOWER_COLON.search(key):
                    key_segments = key.split(':')
                    temp_attrib['type'] = key_segments[0]
                    temp_attrib['key'] = ':'.join(key_segments[1:])
                else:
                    temp_attrib['key'] = key
                temp_attrib['id'] = way_attribs['id']
                temp_attrib['type'] = default_tag_type
                temp_attrib['value'] = child.get('v')
                tags.append(temp_attrib)
            if child.tag == 'nd':
                temp_attrib['id'] = way_attribs['id']
                temp_attrib['node_id'] = child.get('ref')
                temp_attrib['position'] = i
                way_nodes.append(temp_attrib)
                i += 1
   
    # Clean the element's tags
    for tag in tags:
        
        # Expand abbreviations in address fields
        if (tag['key'] == 'address') or \
           (tag['key'] == 'addr' and 'street' in tag['type']):
            tag['value'] = fix_street_abbrevs(tag['value'])
        
        # Replace invalid zipcodes with 'fixme'
        if (tag['key'] in ('zip_left', 'zip_right')) or \
           (tag['key'] == 'addr' and tag['type'] == 'postcode'):
            tag['value'] = fix_zipcode(tag['value'])
            
        
        
    
    
    # Shape the element for integration into the database            
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

KeyboardInterrupt: 

In [31]:
'street' in 'street_1'

True